In [1]:
%cd /home/jovyan/work

from bokeh.io import output_notebook

output_notebook()


/home/jovyan/work


Loading BokehJS ...

In [2]:
from bokeh.io import show
from bokeh.plotting import figure
from bokeh.util.hex import axial_to_cartesian
from bokeh.models import HoverTool

from chordifier.KeyboardRenderer import KeyboardRenderer
from chordifier.algorithm.Sequencer import Sequencer

import numpy as np
import pprint

np.set_printoptions(precision=4, suppress=True)
pp = pprint.PrettyPrinter(indent=4)


def present(keyboard):
    renderer = KeyboardRenderer(keyboard)

    f = make_figure()
    draw_hexagons(f, renderer.qs, renderer.rs, renderer.colors, renderer.positions)
    return f

def make_figure():
   
    f = figure(tools="pan,wheel_zoom,reset", height=600,
               sizing_mode='stretch_width', match_aspect=True)
    f.grid.visible = False
    quad = f.quad(top=[15], bottom=[-16], left=[-25],
       right=[9], color="#000000", alpha=0)
    
    hover = HoverTool()
    hover.tooltips = [
        ("x", "$x"),
        ("y", "$y"),
    ]
    hover.renderers = [quad]
    hover.point_policy = "follow_mouse"
    hover.attachment = "right"
    f.add_tools(hover)

    return f

def draw_hexagons(f, q, r, colors, labels):
    f.hex_tile(q, r, fill_color=colors, alpha=0.5, line_color="white",
               orientation="flattop")

    x, y = axial_to_cartesian(q, r, 1, "flattop")

    f.text(x, y, text=labels,
           text_baseline="middle", text_align="center")


In [3]:
from chordifier.algorithm.Pipeline import Pipeline
from chordifier.experiments.KeyboardFactory import make
from chordifier.utils import vector

PARAMETERS = {
    # general
    'keyboard': make([2, 2, 2, 2, 2] + [0] * 5),
    'characters': 15,

    # sequencer
    'filename': "dataset/ngram.json",
    'length': 3,
    'samples': 500,

    # dynamics
    'x_y_ratio': 1.5,
    'stiffness': vector(1.8, 1.3, 1.2, 1.1, 2, 2, 1.1, 1.2, 1.3, 1.8),

    # pruner
    'priority': vector(18000, 13000, 12000, 11000, 14500,
                       14516, 11008, 12004, 13002, 18001),
    'finger_priorities': 0.1,
    'average_offsets': 0.05,
    'deviation_of_offsets': 0.2,

    # evaluator
    'distances_travelled': 1,
    'chord_difficulties': 1,

    # optimizer
    'best': False,
    'generations': 5,
    'population_size': 100,
    'mate_probability': 0.5,
    'mutate_probability': 0.20,
    'mutate_independent_probability': 0.10,
    'select_tournament_size': 10,
}

pipeline = Pipeline(PARAMETERS)

pipeline.prepare()
result = pipeline.optimize()


gen	nevals	average    	standard   	minimum    	maximum    
0  	100   	2.20343e+08	1.03436e+07	2.00951e+08	2.45558e+08
1  	53    	2.09126e+08	7.21695e+06	1.95223e+08	2.30739e+08
2  	60    	2.08264e+08	1.0905e+07 	1.89912e+08	2.39632e+08
3  	48    	2.00722e+08	1.09503e+07	1.83139e+08	2.43212e+08
4  	51    	1.98042e+08	1.24628e+07	1.83139e+08	2.50318e+08
5  	61    	1.91417e+08	1.01541e+07	1.81734e+08	2.33211e+08


In [4]:
s = Sequencer("dataset/ngram.json", 4, 3, 9)

print("=== Sequencer test ===")
print("--- sequences ---")
print(s.sequences)
print("--- occurrences ---")
print(s.occurrences)
print("--- uniques ---")
print(s.uniques)
print("--- indices ---")
print(s.indices)


=== Sequencer test ===
--- sequences ---
[['e' 't' '_']
 ['e' 't' ' ']
 ['t' 'e' '_']
 ['e' ' ' 't']
 ['t' 'e' ' ']
 ['t' ' ' 't']
 ['_' 't' ' ']
 [' ' '_' '_']
 ['e' '_' 't']]
--- occurrences ---
[766134 519918 484094 484074 435720 308600 248276 246470 222423]
--- uniques ---
[' ' '_' 'e' 't']
--- indices ---
[[2 3 1]
 [2 3 0]
 [3 2 1]
 [2 0 3]
 [3 2 0]
 [3 0 3]
 [1 3 0]
 [0 1 1]
 [2 1 3]]


In [5]:
print("=== Sequencer ===")
print("--- sequences ---")
print(pipeline.sequencer.sequences[:10])
print("--- occurrences ---")
print(pipeline.sequencer.occurrences[:10])
print("--- uniques ---")
print(pipeline.sequencer.uniques)
print("--- indices ---")
print(pipeline.sequencer.indices[:10])

=== Sequencer ===
--- sequences ---
[['i' 'n' 'e']
 ['n' 'e' ' ']
 ['s' 't' 'r']
 ['s' 't' 'a']
 ['i' 'n' 't']
 ['c' 't' ' ']
 ['t' 'a' 't']
 ['\n' 's' 't']
 ['c' 'o' 'n']
 ['_' 'r' 'e']]
--- occurrences ---
[2576070 2408500 2159460 1958257 1852123 1825148 1592096 1517714 1488500
 1438328]
--- uniques ---
['\n' ' ' '_' 'a' 'c' 'd' 'e' 'i' 'l' 'n' 'o' 'p' 'r' 's' 't']
--- indices ---
[[ 7  9  6]
 [ 9  6  1]
 [13 14 12]
 [13 14  3]
 [ 7  9 14]
 [ 4 14  1]
 [14  3 14]
 [ 0 13 14]
 [ 4 10  9]
 [ 2 12  6]]


In [6]:
print("=== Keyboard ===")
f = present(pipeline.keyboard)
show(f)


=== Keyboard ===


In [7]:
print("=== Preprocessor ===")
print("--- zones ---")
for zone in pipeline.preprocessor.zones:
    print(zone)
print("--- chords ---")
print(pipeline.preprocessor.chords[222:999:666])
print("--- positions ---")
print(pipeline.preprocessor.positions[222:999:666])
print("--- origins ---")
print(pipeline.preprocessor.origins)


=== Preprocessor ===
--- zones ---
[[-8  4]
 [-8  3]
 [-9  4]]
[[-7  3]
 [-7  2]]
[[-6  2]
 [-6  1]]
[[-5  2]
 [-5  1]
 [-4  1]]
[[-3  3]
 [-3  2]
 [-2  2]
 [-2  3]
 [-3  4]
 [-4  4]
 [-4  3]]
[]
[]
[]
[]
[]
--- chords ---
[[0 2 0 3 7 0 0 0 0 0]
 [3 0 0 3 1 0 0 0 0 0]]
--- positions ---
[[[     nan      nan]
  [-10.5      2.5981]
  [     nan      nan]
  [ -6.       1.7321]
  [ -6.      -1.7321]
  [     nan      nan]
  [     nan      nan]
  [     nan      nan]
  [     nan      nan]
  [     nan      nan]]

 [[-13.5      0.866 ]
  [     nan      nan]
  [     nan      nan]
  [ -6.       1.7321]
  [ -4.5     -2.5981]
  [     nan      nan]
  [     nan      nan]
  [     nan      nan]
  [     nan      nan]
  [     nan      nan]]]
--- origins ---
[[-12.      -0.    ]
 [-10.5      0.866 ]
 [ -9.       1.7321]
 [ -7.5      0.866 ]
 [ -4.5     -2.5981]
 [     nan      nan]
 [     nan      nan]
 [     nan      nan]
 [     nan      nan]
 [     nan      nan]]


In [8]:
print("=== Pruner intact ===")
print("--- chords ---")
print(pipeline.pruner_intact.chords[222:999:666])
print("--- positions ---")
print(pipeline.pruner_intact.positions[222:999:666])
print("--- origins ---")
print(pipeline.pruner_intact.origins)
print("--- metrics ---")
print(pipeline.pruner_intact.metrics[222:999:666])
print("--- scores ---")
print(pipeline.pruner_intact.scores[222:999:666])
print("--- totals ---")
print(pipeline.pruner_intact.totals[222:999:666])


=== Pruner intact ===
--- chords ---
[[0 2 0 3 7 0 0 0 0 0]
 [3 0 0 3 1 0 0 0 0 0]]
--- positions ---
[[[     nan      nan]
  [-15.75     2.5981]
  [     nan      nan]
  [ -9.       1.7321]
  [ -9.      -1.7321]
  [     nan      nan]
  [     nan      nan]
  [     nan      nan]
  [     nan      nan]
  [     nan      nan]]

 [[-20.25     0.866 ]
  [     nan      nan]
  [     nan      nan]
  [ -9.       1.7321]
  [ -6.75    -2.5981]
  [     nan      nan]
  [     nan      nan]
  [     nan      nan]
  [     nan      nan]
  [     nan      nan]]]
--- origins ---
[[-18.      -0.    ]
 [-15.75     0.866 ]
 [-13.5      1.7321]
 [-11.25     0.866 ]
 [ -6.75    -2.5981]
 [     nan      nan]
 [     nan      nan]
 [     nan      nan]
 [     nan      nan]
 [     nan      nan]]
--- metrics ---
[[38500.         3.2418     1.8819]
 [43500.         2.3305     1.8819]]
--- scores ---
[[0.0562 0.0336 0.1561]
 [0.0635 0.0242 0.1561]]
--- totals ---
[0.2459 0.2438]


In [9]:
print("=== Pruner ===")
print("--- chords ---")
print(pipeline.pruner.chords)
print("--- scores ---")
print(pipeline.pruner.scores)
print("--- totals ---")
print(pipeline.pruner.totals)


=== Pruner ===
--- chords ---
[[3 0 0 0 4 0 0 0 0 0]
 [3 2 2 3 4 0 0 0 0 0]
 [3 2 2 3 6 0 0 0 0 0]
 [3 0 0 3 4 0 0 0 0 0]
 [3 0 0 3 6 0 0 0 0 0]
 [3 2 0 3 6 0 0 0 0 0]
 [3 2 0 3 4 0 0 0 0 0]
 [3 0 2 3 4 0 0 0 0 0]
 [3 0 2 3 6 0 0 0 0 0]
 [3 2 2 3 5 0 0 0 0 0]
 [3 2 1 3 4 0 0 0 0 0]
 [3 2 1 3 6 0 0 0 0 0]
 [3 1 2 3 6 0 0 0 0 0]
 [3 1 2 3 4 0 0 0 0 0]
 [3 2 0 3 5 0 0 0 0 0]]
--- scores ---
[[0.0474 0.0475 0.2   ]
 [0.1    0.0335 0.1603]
 [0.1    0.0335 0.1603]
 [0.0635 0.0408 0.1886]
 [0.0635 0.0408 0.1886]
 [0.0825 0.0365 0.1734]
 [0.0825 0.0365 0.1734]
 [0.081  0.036  0.1734]
 [0.081  0.036  0.1734]
 [0.1    0.0307 0.1584]
 [0.1    0.0292 0.1577]
 [0.1    0.0292 0.1577]
 [0.1    0.0288 0.1577]
 [0.1    0.0288 0.1577]
 [0.0825 0.0329 0.1704]]
--- totals ---
[0.2949 0.2938 0.2938 0.2929 0.2929 0.2924 0.2924 0.2905 0.2905 0.289
 0.2869 0.2869 0.2865 0.2865 0.2858]


In [10]:
permutation = np.arange(0,15)[::-1]
total = pipeline.evaluator.evaluate(permutation)

print("=== Evaluator ===")
print("--- pruner_permuted.chords ---")
print(pipeline.evaluator.pruner_permuted.chords)
print("--- pruner_permuted.positions ---")
#print(pipeline.evaluator.pruner_permuted.positions)
print("--- pruner_permuted.scores ---")
print(pipeline.evaluator.pruner_permuted.scores)
print("--- chords ---")
print(pipeline.evaluator.chords[:2])
print("--- positions ---")
#print(pipeline.evaluator.positions[:2])
print("--- metrics ---")
print(pipeline.evaluator.metrics[:2])
print("--- scores ---")
print(pipeline.evaluator.scores[:2])
print("--- totals ---")
print(pipeline.evaluator.totals[:2])
print("--- occurrence_weighted ---")
print(pipeline.evaluator.occurrence_weighted[:2])
print("--- total ---")
print(total[0]) 
print("--- retrieve_mapping ---")
pp.pprint(pipeline.evaluator.retrieve_mapping(permutation))


=== Evaluator ===
--- pruner_permuted.chords ---
[[3 2 0 3 5 0 0 0 0 0]
 [3 1 2 3 4 0 0 0 0 0]
 [3 1 2 3 6 0 0 0 0 0]
 [3 2 1 3 6 0 0 0 0 0]
 [3 2 1 3 4 0 0 0 0 0]
 [3 2 2 3 5 0 0 0 0 0]
 [3 0 2 3 6 0 0 0 0 0]
 [3 0 2 3 4 0 0 0 0 0]
 [3 2 0 3 4 0 0 0 0 0]
 [3 2 0 3 6 0 0 0 0 0]
 [3 0 0 3 6 0 0 0 0 0]
 [3 0 0 3 4 0 0 0 0 0]
 [3 2 2 3 6 0 0 0 0 0]
 [3 2 2 3 4 0 0 0 0 0]
 [3 0 0 0 4 0 0 0 0 0]]
--- pruner_permuted.positions ---
--- pruner_permuted.scores ---
[[0.0825 0.0329 0.1704]
 [0.1    0.0288 0.1577]
 [0.1    0.0288 0.1577]
 [0.1    0.0292 0.1577]
 [0.1    0.0292 0.1577]
 [0.1    0.0307 0.1584]
 [0.081  0.036  0.1734]
 [0.081  0.036  0.1734]
 [0.0825 0.0365 0.1734]
 [0.0825 0.0365 0.1734]
 [0.0635 0.0408 0.1886]
 [0.0635 0.0408 0.1886]
 [0.1    0.0335 0.1603]
 [0.1    0.0335 0.1603]
 [0.0474 0.0475 0.2   ]]
--- chords ---
[[[1 1 1 1 1 1 1 1 1 1]
  [3 0 2 3 6 0 0 0 0 0]
  [3 0 2 3 4 0 0 0 0 0]
  [3 2 0 3 6 0 0 0 0 0]
  [3 0 2 3 6 0 0 0 0 0]]

 [[1 1 1 1 1 1 1 1 1 1]
  [3 1 2 3 4 0 0 0

In [11]:
from chordifier.algorithm.Evaluator import determine_source_positions

positions = np.arange(1*5*10*2).reshape(1, 5, 10, 2)

chords = np.array([[
    [1,1,1,1,1,1,1,1,1,1],
    [0,0,0,3,7,0,0,0,0,0],
    [0,2,1,3,5,0,0,0,0,0],
    [0,0,2,3,0,0,0,0,0,0],
    [0,0,0,3,7,0,0,0,0,0],
]])

source_positions = determine_source_positions(chords, positions)

print("=== determine_source_positions ===")
print("--- positions ---")
print(positions)
print("--- chords ---")
print(chords)
print("--- source_positions ---")
print(source_positions)

=== determine_source_positions ===
--- positions ---
[[[[ 0  1]
   [ 2  3]
   [ 4  5]
   [ 6  7]
   [ 8  9]
   [10 11]
   [12 13]
   [14 15]
   [16 17]
   [18 19]]

  [[20 21]
   [22 23]
   [24 25]
   [26 27]
   [28 29]
   [30 31]
   [32 33]
   [34 35]
   [36 37]
   [38 39]]

  [[40 41]
   [42 43]
   [44 45]
   [46 47]
   [48 49]
   [50 51]
   [52 53]
   [54 55]
   [56 57]
   [58 59]]

  [[60 61]
   [62 63]
   [64 65]
   [66 67]
   [68 69]
   [70 71]
   [72 73]
   [74 75]
   [76 77]
   [78 79]]

  [[80 81]
   [82 83]
   [84 85]
   [86 87]
   [88 89]
   [90 91]
   [92 93]
   [94 95]
   [96 97]
   [98 99]]]]
--- chords ---
[[[1 1 1 1 1 1 1 1 1 1]
  [0 0 0 3 7 0 0 0 0 0]
  [0 2 1 3 5 0 0 0 0 0]
  [0 0 2 3 0 0 0 0 0 0]
  [0 0 0 3 7 0 0 0 0 0]]]
--- source_positions ---
[[[ 0  1]
  [42 43]
  [64 65]
  [66 67]
  [48 49]
  [10 11]
  [12 13]
  [14 15]
  [16 17]
  [18 19]]]
